# MCP Server

## Introduction

In this notebook, we will demonstrate the use of the MCP server workflow.
This workflow runs a server that provides access to an ApertureDB instance via the Model Context Protocol (MCP).

## Setup

In order to run this notebook, you will need to be running the MCP server workflow.
You can do this conveniently in the Cloud UI.
You will also need to know the authorization bearer token for the MCP server.


In [1]:
from fastmcp import Client
from getpass import getpass
import os
import json

## Enter Authorization Bearer Token

In [2]:
TOKEN = getpass("authorization bearer token")

## Set up the client

The FastMCP library is commonly used to create MCP servers,
but here we use the client feature.

In [ ]:
URL = 'https://<DB_HOST>/mcp/'
client = Client(URL, auth=TOKEN)

## Test the connection

If everything is set up correctly, this will print `True`.

In [4]:
async with client:
    print(await client.ping())

True


## List tools

Let's find out what tools the server has.

In [ ]:
async with client:
    tools = await client.list_tools()
    for tool in tools:
        print(f"Tool: {tool.name} - {tool.description}")
        print(f"  Parameters: {json.dumps(tool.inputSchema, indent=2, ensure_ascii=False)}")
        print()

## Run a tool: List Descriptor Sets

Let's find out what descriptor sets are in our system.

In [ ]:
async with client:
    descriptor_sets = await client.call_tool("list_descriptor_sets")
print(descriptor_sets.data.sets)
descriptor_set = [
    ds.name for ds in descriptor_sets.data.sets if ds.count > 0][0]
print(descriptor_set)

## Run a tool: Find Similar Documents

Here we run the `find_similar_documents` tool against the default descriptor set and print the result documents.

In [ ]:
text = "Find documents similar to this text."
async with client:
    docs = await client.call_tool("find_similar_documents", dict(query=text, descriptor_set=descriptor_set))
    for doc in docs.data.documents:
        print(f"* {doc.url}: {doc.text}")

## Run a tool: List Entity Classes

What types of entity are there?

In [ ]:
async with client:
    results = await client.call_tool("list_entity_classes")
classes = json.loads(results.content[0].text)
print(json.dumps(classes, indent=2, ensure_ascii=False))
entity_class = "_Image" if "Image" in classes else classes[0]

## Run a tool: Count images

How many images are there in the database? We can see the `matched` field here.

In [ ]:
print(f"Looking for class {entity_class}")
async with client:
    description = await client.call_tool('describe_entity_class',
                                         dict(class_name=entity_class))
    print(json.dumps(json.loads(
        description.content[0].text), indent=2, ensure_ascii=False))